# L’écart avec l’espérance comme expression de la distance entre vecteurs

L’idée derrière la notion d’écart avec l’espérance, en traitement automatique du langage, consiste à comparer la fréquence des mots d’un texte avec celles attendues, calculées à partir de corpus de référence. Il serait ainsi possible de déterminer si telle pièce a bien été écrite par Molière ou s’il s’agit plutôt d’une œuvre non assumée de Corneille, si telle autre relève plutôt du tragique ou du comique, ou si un tweet est une anarque ou bien un fait avéré.

La méthode retenue appartient au domaine de la statistique. Elle a été proposée par Adam Kilgarriff dans un article intitulé "Comparing Corpora" et publié en 2001 dans le [*International Journal of Corpus Linguistics*](https://doi.org/10.1075/ijcl.6.1.05kil). En convoquant le test du $\chi^2$ qui d’ordinaire permet de vérifier si une variable suit une loi du $\chi^2$ sous l’hypothèse nulle, il souhaite estimer la distance qui sépare deux vocabulaires.

## Le test du $\chi^2$ d’indépendance

### Des hypothèses statistiques

Communément, un test statistique sert à vérifier qu’une hypothèse est vraie : c’est *l’hypothèse nulle*, notée $H_0$. Dans le cas contraire, *l’hypothèse alternative*, notée $H_1$, est retenue car considérée comme statistiquement significative. Pour juger de la significativité du test, le résultat de l’expérience est confronté à une loi de probabilité avec un seuil de risque $\alpha$ généralement établi à 0,05 (5 % de risque d’erreur). Si la probabilité d’obtenir une valeur aussi extrême que celle observée, la valeur-p (ou *p-value* en anglais), est inférieure à ce seuil, de fortes présomptions pèsent contre l’hypothèse nulle que l’on vient de tester. À l’inverse, si la valeur-p est supérieure à ce seuil, on conclut simplement à l’échec du test, sans ne rien dire des hypothèses formulées.

### Principe du test d’indépendance

L’une des applications connues du test du $\chi^2$ intervient dans l’estimation de la liaison entre deux variables qualitatives : soit elles sont indépendantes ($H_0$) soit elles covarient de manière statistiquement significative ($H_1$).

Pour parvenir au résultat, la méthode consiste à établir, pour les variables ciblées, la distance entre les valeurs empiriques ($O_{ij}$) et les valeurs théoriques normalement attendues ($E_{ij}$) si elles étaient indépendantes, selon la formule :

$$
\chi^2 = \sum_{i,j} \frac{(O_{ij} - E_{ij})^2}{E_{ij}}
$$

La somme obtenue est alors comparée à [la loi du $\chi^2$](https://fr.wikipedia.org/wiki/Loi_du_%CF%87%C2%B2#Table_de_valeurs_des_quantiles) avec $k$ degrés de liberté pour un risque d’erreur de 5 %, sachant que les degrés de liberté s’obtiennent en effectuant le produit entre le nombre de lignes et le nombre de colonnes selon la formule :

$$
k = (I – 1)(J – 1)
$$

### Comment survivre au naufrage du *Titanic*

Émettons l’hypothèse qu’être riche nous aurait fourni plus de chances pour survivre au naufrage du *Titanic*. Mais comment s’en assurer ? Mobilisons [un jeu de données](./data/titanic.csv) qui fournit quelques informations sur les passagers du fameux paquebot :

In [ ]:
import pandas as pd

df = pd.read_csv('./data/titanic.csv')

display(df.head())

Retenons les variables `Pclass` et `Survived`, la première indiquant la classe de transport (1e, 2e ou 3e classe) et la seconde nous apprenant si la personne a survécu ou non (1 ou 0) :

In [ ]:
data = df[["Survived", "Pclass"]]

display(data.head())

Créons maintenant une table de contingence afin de savoir combien de passagers et passagères ont péri ou survécu en fonction de leurs conditions d’hébergement à bord :

In [ ]:
contingency = pd.crosstab(df.Pclass, df.Survived)

display(contingency)

Dans ce tableau, nous lisons par exemple que 80 personnes de la première classe ont péri contre 372 de la seconde classe. À titre informatif, notons qu’il est possible d’obtenir les sommes marginales :

In [ ]:
pd.crosstab(df.Pclass, df.Survived, margins=True)

De là, nous devons calculer les valeurs théoriques attendues ($E_{ij}$) en cas d’indépendance des variables :

$$
E_{ij} = \frac{ \sum_{j=1}^{J}O_{ij} \cdot \sum_{i=1}^{I}O_{ij}}{N}
$$

|Pclass|0|1|
|:-:|:-:|:-|
|1|(216 * 549) / 891 = **133.09**|(216 * 342) / 891 = **82.91**|
|2|(184 * 549) / 891 = **113.37**|(184 * 342) / 891 = **70.63**|
|3|(491 * 549) / 891 = **302.54**|(491 * 342) / 891 = **188.46**|

La matrice $E_{ij}$ peut s’obtenir directement avec Python :

In [ ]:
N = contingency.sum().sum()
Oi = contingency.sum(axis=1).values
Oj = contingency.sum(axis=0).values

# reshape Oi
Oi = Oi.reshape(contingency.shape[0], 1)

Eij = (Oi * Oj) / N

display(Eij)

Il ne reste plus qu’à calculer la valeur du $\chi^2$ :

In [ ]:
chi_squared = (contingency.values - Eij) ** 2 / Eij

display(chi_squared.sum())

#### Interprétation avec la table de distribution du $\chi^2$

La [loi du $\chi^2$](https://fr.wikipedia.org/wiki/Loi_du_%CF%87%C2%B2#Table_de_valeurs_des_quantiles) fournit des valeurs critiques pour chaque niveau de seuil $\alpha$ avec $k$ degrés de liberté. Dans notre exemple, nous avons retenu un seuil de risque de 0,05 et le nombre de degrés de liberté est de $(3 - 1)(2 - 1) = 2$. La valeur critique donnée par la table est de 5,99. Notre résultat lui étant bien supérieur, nous pouvons rejeter $H_0$.

#### Interprétation avec la valeur-p

Plus communément, l’interprétation d’un test statistique se fait avec la valeur-p qui dans notre exemple exprime la probabilité d’obtenir une valeur aussi extrême que celle observée. Si la valeur-p est inférieure au seuil de 0,05, nous pouvons rejeter l’hypothèse nulle.

La méthode `chi2.sf()` de *Scipy* permet de calculer la valeur-p à partir d’un $\chi^2$ et du nombre de degrés de libertés :

In [ ]:
from scipy.stats import chi2

display(chi2.sf(chi_squared.sum(), 2))

À noter l’existence d’une fonction `chi2_contingency()` qui ressort toutes les mesures désirées à partir d’un tableau de contingence sans les sommes marginales :

In [ ]:
from scipy.stats import chi2_contingency

chi2, pvalue, degrees, expected = chi2_contingency(contingency)

print(
    f"Valeur du khi-deux : {chi2:.4f}",
    f"Probabilité du khi-deux (valeur-p) : {pvalue:.4f}",
    sep="\n"
)

## Test de la distance entre deux vocabulaires

Formulons une nouvelle hypothèse selon laquelle un texte anonyme n’est pas de la main d’un auteur particulier. Pour la tester, nous envisageons de calculer la distance entre la fréquence des occurrences des mots dans le texte anonyme et celle observée dans le corpus de l’auteur. Si à la fin du test la valeur-p est inférieure au seuil de 5 %, nous présumons que notre hypothèse est fausse et retenons l’hypothèse alternative.

La méthode consiste à :

1. obtenir les vocabulaires du texte anonyme et du corpus de l’auteur ;
2. les combiner afin d’obtenir un vocabulaire complet ;
3. sélectionner les $n$ mots les plus fréquents du corpus combiné ;
4. calculer les valeurs théoriques ($E_{ij}$) dans chacun des deux corpus ;
5. calculer la distance avec les valeurs empiriques.

Avant de commencer, chargeons toutes les bibliothèques logicielles nécessaires ainsi que les textes à comparer :

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')

corpus = [
    "Le petit chat boit du lait.",
    "Le petit chien boit de l’eau.",
    "La vache boit de l’eau mais ne boit pas de lait."
]

anonym = "Le petit oiseau boit de l’eau."

### Étape 1 : obtenir les vocabulaires

La réalisation de cette étape consiste à appliquer toutes les opérations mises en jeu dans une tâche de sac de mots. Nous nous contentons ici d’une banale tokenisation afin d’obtenir au final deux matrices d’occurrences.

#### La matrice des occurrences du corpus

La segmentation est effectuée pour chaque texte de la variable `corpus` en filtrant sur les mots de deux caractères et moins :

In [ ]:
words_corpus = [
    word.lower()
    for sent in corpus
    for word in tokenizer.tokenize(sent)
    if len(word) > 2
]

Grâce à un objet de type `Counter()`, il est maintenant aisé d’obtenir les fréquences d’apparition des mots dans le corpus :

In [ ]:
occurrences_corpus = Counter(words_corpus)

#### La matrice d’occurrences du texte anonyme

Répétons l’opération pour le texte anonyme :

In [ ]:
occurrences_anonym = Counter([
    word.lower()
    for word in tokenizer.tokenize(anonym)
    if len(word) > 2
])

### Étape 2 : combiner les deux vocabulaires

Une opération triviale qui consiste à fusionner les deux objets de type `Counter` en un troisième :

In [ ]:
occurrences_all = occurrences_corpus + occurrences_anonym

Regardons un aperçu du résultat dans un *data frame* :

In [ ]:
pd.DataFrame(
    data=occurrences_all.values(),
    index=occurrences_all.keys(),
    columns=["All"]
)